# Revisiting the NYC Taxi DataSet Model Architecture Part 3

In this notebook, the model architecture is being changed in order to understand the correlation between the model architecture and the prediction quality.
This model uses sequences of locations as inputs for the GRU network component.
That component consists of two GRU layers which are then directly going in a Dense Layer with SoftMax activation to obtain the logits.
The deep learning problem can thus be described as follows:
Predict the (N+1)th location a history of prior locations in form a sequence of N locations.
This model is also based on the changed network architecture from Part 1.
As a result, it only predicts the relevant (N+1)th location and not all next locations for every sequence element like it was in the original architecture.


In [1]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


# Dataset

In [3]:
df = pd.read_csv("./ma_results/trips_with_zones_final.csv")
df = df.head(10000000)
df.head(10)

,medallion,pickup_week_day,pickup_hour,pickup_day,pickup_month,dropoff_week_day,dropoff_hour,dropoff_day,dropoff_month,pickup_location_id,dropoff_location_id
0,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,0,1,1,162.0,262.0
1,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,0,1,1,262.0,239.0
2,00005007A9F30E289E760362F69E4EAD,1,0,1,1,1,1,1,1,239.0,236.0
3,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,1,1,1,236.0,41.0
4,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,1,1,1,41.0,211.0
5,00005007A9F30E289E760362F69E4EAD,1,1,1,1,1,2,1,1,211.0,238.0
6,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,2,1,1,238.0,142.0
7,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,2,1,1,142.0,263.0
8,00005007A9F30E289E760362F69E4EAD,1,2,1,1,1,3,1,1,263.0,48.0
9,00005007A9F30E289E760362F69E4EAD,1,3,1,1,1,3,1,1,48.0,246.0


In [4]:
# Check dtypes of the attributes
df.dtypes

medallion               object
pickup_week_day          int64
pickup_hour              int64
pickup_day               int64
pickup_month             int64
dropoff_week_day         int64
dropoff_hour             int64
dropoff_day              int64
dropoff_month            int64
pickup_location_id     float64
dropoff_location_id    float64
dtype: object

In [5]:
# Drop the medallion, it is not needed for this example
df.drop(['medallion'], axis=1, inplace=True)

Because there are too many taxis (over 9000) it is better to take the 100 taxi with the major number of records

In [6]:
# Cast the columns type to int32
dictionary = {'pickup_week_day': 'int32', 'pickup_hour': 'int32', 'pickup_day': 'int32', 'pickup_month': 'int32', 'dropoff_week_day': 'int32', 'dropoff_hour': 'int32', 'dropoff_day': 'int32', 'dropoff_month': 'int32', 'pickup_location_id':'int32', 'dropoff_location_id':'int32'}
df = df.astype(dictionary, copy=True)
df.dtypes

pickup_week_day        int32
pickup_hour            int32
pickup_day             int32
pickup_month           int32
dropoff_week_day       int32
dropoff_hour           int32
dropoff_day            int32
dropoff_month          int32
pickup_location_id     int32
dropoff_location_id    int32
dtype: object

We can use the other taxis to create a local test and validation sets

Now we need to create the location sequence for each user

In [7]:
mh = ModelHelper(df, 129)

In [8]:
# Call the function
mh.df_to_location_sequence()

print(mh.df)

            index  location_id  day  month  hour_sin      hour_cos  \
0               0          162    1      1  0.000000  1.000000e+00   
1              12          230    1      1  0.707107  7.071068e-01   
2              13          125    1      1  0.707107  7.071068e-01   
3              15           48    1      1  0.866025  5.000000e-01   
4              18          170    1      1  1.000000  6.123234e-17   
...           ...          ...  ...    ...       ...           ...   
13731996  7284341          161   26      1 -0.500000 -8.660254e-01   
13731997  7284341          161   26      1 -0.500000 -8.660254e-01   
13731998  7284342          132   26      1 -0.707107 -7.071068e-01   
13731999  7284343          141   26      1 -0.866025 -5.000000e-01   
13732000  7284344          141   26      1 -0.866025 -5.000000e-01   

          week_day_sin  week_day_cos  weekend  
0             0.781831      0.623490        0  
1             0.781831      0.623490        0  
2             0

In [9]:
mh.train_val_test_split()
print(len(mh.df_train), 'train examples')
print(len(mh.df_val), 'validation examples')
print(len(mh.df_test), 'test examples')

8788480 train examples
2197120 validation examples
2746401 test examples


In [10]:
mh.split_data()
mh.list_test[0]

,index,location_id,day,month,hour_sin,hour_cos,week_day_sin,week_day_cos,weekend
10985600,5283998,246,4,1,-0.866025,0.500000,-0.433884,-0.900969,0
10985601,5283999,107,4,1,-0.707107,0.707107,-0.433884,-0.900969,0
10985602,5284000,142,4,1,-0.707107,0.707107,-0.433884,-0.900969,0
10985603,5284001,48,4,1,-0.500000,0.866025,-0.433884,-0.900969,0
10985604,5284001,48,4,1,-0.500000,0.866025,-0.433884,-0.900969,0
...,...,...,...,...,...,...,...,...,...
10985724,5284091,234,7,1,0.000000,1.000000,0.000000,1.000000,0
10985725,5284092,162,7,1,0.258819,0.965926,0.000000,1.000000,0
10985726,5284093,142,7,1,0.500000,0.866025,0.000000,1.000000,0
10985727,5284093,142,7,1,0.500000,0.866025,0.000000,1.000000,0


In [11]:
mh.set_batch_size(128)
mh.create_batch_dataset()
mh.test_dataset

<BatchDataset shapes: ({start_place: (128, 128), start_hour_sin: (128, 128), start_hour_cos: (128, 128), weekend: (128, 128), week_day_sin: (128, 128), week_day_cos: (128, 128), end_hour_sin: (128,), end_hour_cos: (128,), end_weekend: (128,), end_week_day_sin: (128,), end_week_day_cos: (128,)}, (128,)), types: ({start_place: tf.int32, start_hour_sin: tf.float64, start_hour_cos: tf.float64, weekend: tf.int32, week_day_sin: tf.float64, week_day_cos: tf.float64, end_hour_sin: tf.float64, end_hour_cos: tf.float64, end_weekend: tf.int32, end_week_day_sin: tf.float64, end_week_day_cos: tf.float64}, tf.int32)>

In [12]:
mh.set_target_column_name('location_id')
mh.set_vocab_size()
mh.set_numerical_column_names(['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day'])

In [13]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 256

# Create a model
def create_model():
  N = mh.total_window_length
  batch_size = mh.batch_size
  number_of_places = mh.vocab_size

	# Shortcut to the layers package
  l = tf.keras.layers

   # Declare the dictionary for the places sequence as before
  sequence_input = {
      'start_place': tf.keras.Input((N-1,), batch_size=batch_size, dtype=tf.dtypes.int32, name='start_place') # add batch_size=batch_size in case of stateful GRU
  }

  # Handling the categorical feature sequence using one-hot
  places_one_hot = feature_column.sequence_categorical_column_with_vocabulary_list(
      'start_place', [i for i in range(number_of_places)])

  # Embed the one-hot encoding
  places_embed = feature_column.embedding_column(places_one_hot, embedding_dim)

  # With an input sequence we can't use the DenseFeature layer, we need to use the SequenceFeatures
  sequence_features, sequence_length = tf.keras.experimental.SequenceFeatures(places_embed)(sequence_input)

  sequence_features = tf.ensure_shape(sequence_features, (batch_size, N-1, sequence_features.shape[2]))

  # Rnn
  recurrent = l.GRU(rnn_units,
                        batch_size=batch_size, #in case of stateful
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(sequence_features)

  recurrent_2 = l.GRU(64,
                        batch_size=batch_size, #in case of stateful
                        stateful=True,
                        recurrent_initializer='glorot_uniform')(recurrent)

	# Last layer with an output for each places
  dense_1 = layers.Dense(number_of_places)(recurrent_2)

	# Softmax output layer
  output = l.Softmax()(dense_1)

	# To return the Model, we need to define its inputs and outputs
	# In out case, we need to list all the input layers we have defined
  inputs = list(sequence_input.values())

	# Return the Model
  return tf.keras.Model(inputs=inputs, outputs=output)

In [14]:
# Get the model and compile it
mh.assign_model(create_model())
mh.compile_model()

# Training

In [15]:
mh.set_num_epochs(10)
mh.fit_model()

Epoch 1/10


C:\Users\WoodPecker\Documents\Privat\HTW-Master\Sem3\PA\HumanMobilityPredictionMA\venv\lib\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['start_hour_sin', 'start_hour_cos', 'weekend', 'week_day_sin', 'week_day_cos', 'end_hour_sin', 'end_hour_cos', 'end_weekend', 'end_week_day_sin', 'end_week_day_cos'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


532/532 [==============================] - 244s 459ms/step - loss: 4.1860 - sparse_categorical_accuracy: 0.0453 - val_loss: 4.0380 - val_sparse_categorical_accuracy: 0.0801
Epoch 2/10
532/532 [==============================] - 231s 434ms/step - loss: 3.9600 - sparse_categorical_accuracy: 0.1225 - val_loss: 3.7722 - val_sparse_categorical_accuracy: 0.2139
Epoch 3/10
532/532 [==============================] - 224s 421ms/step - loss: 3.8242 - sparse_categorical_accuracy: 0.1842 - val_loss: 3.6407 - val_sparse_categorical_accuracy: 0.2492
Epoch 4/10
532/532 [==============================] - 226s 426ms/step - loss: 3.7530 - sparse_categorical_accuracy: 0.2050 - val_loss: 3.5722 - val_sparse_categorical_accuracy: 0.2721
Epoch 5/10
532/532 [==============================] - 230s 433ms/step - loss: 3.7033 - sparse_categorical_accuracy: 0.2158 - val_loss: 3.5373 - val_sparse_categorical_accuracy: 0.2798
Epoch 6/10
532/532 [==============================] - 231s 435ms/step - loss: 3.6665 - spar

# Evaluation

In [16]:
mh.evaluate_model()

166/166 [==============================] - 18s 108ms/step - loss: 3.5253 - sparse_categorical_accuracy: 0.2797


In [17]:
mh.model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
start_place (InputLayer)     [(128, 128)]              0         
_________________________________________________________________
sequence_features (SequenceF ((None, None, 256), (None 67584     
_________________________________________________________________
tf_op_layer_EnsureShape (Ten [(128, 128, 256)]         0         
_________________________________________________________________
gru (GRU)                    (128, 128, 256)           394752    
_________________________________________________________________
gru_1 (GRU)                  (128, 64)                 61824     
_________________________________________________________________
dense (Dense)                (128, 264)                17160     
_________________________________________________________________
softmax (Softmax)            (128, 264)               

In [18]:
mh.print_test_prediction_info()

logits
Shape :  (21248, 264)
Example [0] :  [5.30494784e-04 1.64323923e-04 6.92517332e-08 4.56704402e-06
 1.03461242e-03 2.55441762e-07 4.84571387e-07 1.07182667e-03
 9.85115457e-06 3.44731689e-06 1.59870760e-05 4.05478750e-06
 1.82899064e-04 4.08092467e-03 1.87336598e-04 2.47297976e-06
 2.86690835e-07 7.24800921e-05 5.83591791e-05 2.76320662e-07
 8.42785482e-07 1.09552157e-05 2.28656427e-05 6.63835579e-07
 3.96374753e-03 2.71883589e-04 1.37602219e-05 4.18450554e-08
 1.30673854e-07 1.92502848e-05 2.39567814e-08 1.13823153e-05
 1.01036028e-06 6.29448099e-04 1.15810490e-05 2.31947683e-06
 5.33016864e-05 8.76752893e-05 4.92527943e-06 1.10343180e-05
 1.32319212e-04 5.08598331e-03 1.04681996e-03 1.33284358e-02
 9.71893996e-07 7.85556971e-04 1.07148628e-06 1.32435935e-05
 5.54284304e-02 3.17069382e-04 1.13848895e-02 9.49335390e-06
 2.81587680e-04 1.34139727e-05 2.12952837e-05 6.85843042e-06
 7.39434745e-06 3.85819277e-07 2.08405891e-06 4.52713834e-07
 7.84942858e-06 1.57606759e-04 8.57097402

The assumption proved correct.
Using only the location history and no time components at all results in a similar prediction quality.
That means that the network can predict the next location on the last locations.

Note after evaluating the central model prediction:
The behaviour of the model makes sense as it mostly predicts the user to remain in the same area.
Thus, time components are irrelevant.